In [65]:
import warnings
warnings.filterwarnings("ignore")

import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler, MinMaxScaler
#import wrangle
import env
import seaborn as sns
import acquire
import prep
import scipy.stats as stats

In [2]:
#single family residence from zillow DB
df = prep.prep_df()

In [ ]:
df.isnull().sum().sort_values(ascending=False)

In [ ]:
#drops columns with < 80% of values filled in
df = prep.impute_values(df)
df = prep.drop_columns(df)

In [ ]:
df = df.dropna(how='any')

In [3]:
test = df[(df.propertycountylandusecode != "010G") & ((df.propertycountylandusecode != "010M"))]

test.propertycountylandusecode.astype(float)

0         100.0
1         100.0
2         100.0
3         100.0
4         100.0
5         101.0
6         100.0
7           1.0
8           1.0
9           1.0
13          1.0
14          1.0
15          1.0
16          1.0
17          1.0
18          1.0
19          1.0
30       1110.0
31       1110.0
32       1110.0
33       1110.0
34       1110.0
35       1111.0
36       1111.0
37       1111.0
38       1111.0
39       1110.0
40       1110.0
41       1110.0
42       1110.0
          ...  
55549     100.0
55550     100.0
55551     100.0
55552     100.0
55553     100.0
55554     101.0
55555     101.0
55556     100.0
55557     100.0
55558     100.0
55559     100.0
55560     100.0
55561     100.0
55562     100.0
55563     100.0
55564     100.0
55565     100.0
55566     100.0
55567     101.0
55568     100.0
55569     100.0
55570     100.0
55571     101.0
55572     101.0
55573     100.0
55574     100.0
55575     100.0
55576     100.0
55577     100.0
55578     100.0
Name: propertycountyland

In [ ]:
test = df[(df.propertycountylandusecode == "010G") | ((df.propertycountylandusecode == "010M"))]

test.shape

In [ ]:
df[['landtaxvaluedollarcnt','lotsizesquarefeet']].head()

In [ ]:
df['tax_value_per_foot'] = df.landtaxvaluedollarcnt/df.lotsizesquarefeet
df.tax_value_per_foot.mean()

In [ ]:
(df.structuretaxvaluedollarcnt/df.tax_value).mean()

In [4]:
df.head().T

,0,1,2,3,4
bathrooms,2,4,2,1,2
bedrooms,4,3,3,3,3
sqft,3633,1620,2077,1244,1300
tax_value,296425,847770,646760,169471,233266
fips,6037,6037,6037,6037,6037
latitude,3.456e+07,3.39962e+07,3.4013e+07,3.39536e+07,3.38971e+07
longitude,-1.1817e+08,-1.18438e+08,-1.18479e+08,-1.18084e+08,-1.18103e+08
lotsizesquarefeet,9826,12842.7,6490,6021,4917
propertycountylandusecode,0100,0100,0100,0100,0100
regionidcity,40227,12447,26964,13716,53636


In [ ]:
df.lotsizesquarefeet.head()

In [ ]:
df.landtaxvaluedollarcnt /df.tax_value_per_foot.mean()

In [ ]:
df_nulls_r = pd.DataFrame(df.apply(lambda x: df.shape[1] -x.count(),axis=1))
df_nulls_r['pct_rows_missing'] = df_nulls_r[0] / df.shape[1]
row_drops = df_nulls_r[df_nulls_r['pct_rows_missing'] >.2]
row_drops['column_names'] = row_drops.index
row_drops = list(row_drops.column_names)
#df_new = df.drop(row_drops,axis=0)

df_nulls_r.max()


In [15]:
Q1 = df.quantile(.25)
Q3 = df.quantile(.75)
IQR = Q3-Q1

In [26]:
IQR = Q3-Q1
Q1

bathrooms                     2.000000e+00
bedrooms                      3.000000e+00
sqft                          1.260000e+03
tax_value                     1.874700e+05
fips                          6.037000e+03
latitude                      3.383746e+07
longitude                    -1.184017e+08
lotsizesquarefeet             5.677000e+03
regionidcity                  1.244700e+04
regionidcounty                2.061000e+03
regionidzip                   9.619300e+04
yearbuilt                     1.949000e+03
structuretaxvaluedollarcnt    7.515900e+04
assessmentyear                2.016000e+03
logerror                     -2.443694e-02
Name: 0.25, dtype: float64

In [17]:
df[~((df < (Q1 - 1.5 * IQR)) |(df > (Q3 + 1.5 * IQR))).any(axis=1)]

,bathrooms,bedrooms,sqft,tax_value,fips,latitude,longitude,lotsizesquarefeet,propertycountylandusecode,regionidcity,regionidcounty,regionidzip,yearbuilt,structuretaxvaluedollarcnt,assessmentyear,logerror
1,4.0,3.0,1620.0,847770.0,6037.0,33996200.0,-118438000.0,12842.749037,0100,12447.0,3101.0,96047.0,2011.0,339532.0,2016.0,-0.019546
2,2.0,3.0,2077.0,646760.0,6037.0,34012977.0,-118479243.0,6490.000000,0100,26964.0,3101.0,96152.0,1926.0,210192.0,2016.0,-0.040807
3,1.0,3.0,1244.0,169471.0,6037.0,33953559.0,-118083855.0,6021.000000,0100,13716.0,3101.0,96201.0,1950.0,108040.0,2016.0,0.042412
4,2.0,3.0,1300.0,233266.0,6037.0,33897134.0,-118102953.0,4917.000000,0100,53636.0,3101.0,96193.0,1950.0,77415.0,2016.0,-0.000908
744,2.0,3.0,1766.0,390038.0,6037.0,34217790.0,-118628236.0,9085.000000,0101,12447.0,3101.0,96339.0,1962.0,120205.0,2016.0,0.012474
747,3.0,4.0,2397.0,537750.0,6037.0,34212078.0,-118638939.0,12910.000000,0101,12447.0,3101.0,96339.0,1984.0,274403.0,2016.0,0.005598
748,3.0,4.0,2574.0,533947.0,6037.0,34217005.0,-118643730.0,10095.000000,0101,12447.0,3101.0,96339.0,1981.0,239616.0,2016.0,-0.094197
750,3.0,4.0,2661.0,433253.0,6037.0,34214457.0,-118639462.0,9787.000000,0101,12447.0,3101.0,96339.0,1983.0,258245.0,2016.0,0.049512
751,3.0,4.0,1830.0,435890.0,6037.0,34201279.0,-118623456.0,8280.000000,0100,12447.0,3101.0,96342.0,1960.0,132311.0,2016.0,0.028396
752,2.0,3.0,1358.0,371255.0,6037.0,34206413.0,-118618965.0,7551.000000,0101,12447.0,3101.0,96342.0,1956.0,103875.0,2016.0,0.026857


In [55]:
def outlier(series):
    Q1 = series.quantile(.25)
    Q3 = series.quantile(.75)
    IQR = Q3-Q1
    
    series = series[((series < (Q1 - 3 * IQR)) |(series > (Q3 + 3 * IQR)))]
    return series

In [81]:
def remove_outliers_iqr(df, columns):
    for col in columns:
        q75, q25 = np.percentile(df[col], [75,25])
        ub = 3*stats.iqr(df[col]) + q75
        lb = q25 - 3*stats.iqr(df[col])
        print(len(df[df[col] <= ub]))
        df = df[df[col] <= ub]
        df = df[df[col] >= lb]
    return df

In [82]:
len(remove_outliers_iqr(df,['tax_value','sqft','lotsizesquarefeet','structuretaxvaluedollarcnt']))

52935
52686
49564
48919


48919

In [68]:
df.shape

(54544, 16)

['bathrooms',
 'bedrooms',
 'sqft',
 'tax_value',
 'fips',
 'latitude',
 'longitude',
 'lotsizesquarefeet',
 'propertycountylandusecode',
 'regionidcity',
 'regionidcounty',
 'regionidzip',
 'yearbuilt',
 'structuretaxvaluedollarcnt',
 'assessmentyear',
 'logerror']